In [13]:
import pandas as pd
from sklearn.externals import joblib
import category_encoders as ce
import sys
import os
BinEncoder = joblib.load('BinEncoder.pkl') 
rf = joblib.load('RF_modelcars.pkl')

def predict_car_value(Make_par, Model_par,State_par,Mileage_par,Year_par):
       
    #Crear un dataframe y aplicar el encoder binario
    df = pd.DataFrame([{'Year': Year_par, 'Mileage': Mileage_par, 'State':State_par, 'Make':Make_par, 'Model':Model_par}] ) 
    df = createFeatures(df)
    
    df2 = BinEncoder.transform(df)
    
    value = rf.predict(df2)[0]
    return round(value)


def createFeatures(data):
    stateRate = pd.read_csv("StatesRate.csv")
    data = data.set_index('State').join(stateRate.set_index('State')).reset_index()
    data.Year = 2019-int(data.Year)
    data["MakeModel4"] = data["Make"] + data["Model"].str[:4]
    data=data.drop(['State'], axis=1)
    return data

In [18]:
print (predict_car_value('Ford', 'EscapeFWD','MD','177909','2010'))

15936.0


In [1]:
#!/usr/bin/python
from flask import Flask
from flask_restplus import Api, Resource, fields
from sklearn.externals import joblib
from Model_PriceCars import predict_car_value

app = Flask(__name__)

api = Api(
    app, 
    version='1.0', 
    title='Estimador de precio vehicular',
    description='Desarrollado por: Iván Gómez, Cristian Najera, Natalia Martínez')

ns = api.namespace('Predict', 
     description='Predict price')
   
parser = api.parser()

parser.add_argument(
    'Make', 
    type=str, 
    required=True, 
    help='Car Maker', 
    location='args')

parser.add_argument(
    'Model', 
    type=str, 
    required=True, 
    help='Car Model', 
    location='args')

parser.add_argument(
    'State', 
    type=str, 
    required=True, 
    help='State', 
    location='args')

parser.add_argument(
    'Mileage', 
    type=str, 
    required=True, 
    help='Mileage', 
    location='args')

parser.add_argument(
    'Year', 
    type=str, 
    required=True, 
    help='Year', 
    location='args')


resource_fields = api.model('Resource', {
    'result': fields.String,
})

@ns.route('/')
class PredPriceApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        return {
         "result": predict_car_value(args['Make'],args['Model'],args['State'],args['Mileage'],args['Year'])
        }, 200
    
    
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=8889)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:8889/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Mar/2019 19:58:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2019 19:58:26] "GET /swagger.json HTTP/1.1" 200 -
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
127.0.0.1 - - [10/Mar/2019 19:58:54] "GET /Predict/?Make=Ford&Model=EscapeFWD&State=OH&Mileage=23388&Year=2015 HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2019 19:58:54] "GET /favicon.ico HTTP/1.1" 404 -
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
127.0.0.1 - - [10/Mar/2019 19:59:04] "GET /Predict/?Make=Nissan&Model=MuranoAWD&State=MD&Mileage=31909&Year=2014 HTTP/1.1" 200 -
